#Load data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dialogues_018.json to dialogues_018.json


In [2]:
# intents
!mkdir -p dialogue_018/NONE
!mkdir -p dialogue_018/SearchOnewayFlight
!mkdir -p dialogue_018/ReserveOnewayFlight
!mkdir -p dialogue_018/SearchRoundtripFlights
!mkdir -p dialogue_018/ReserveRoundtripFlights

Script para coletar apenas as utterances (frases) do usuário e armazená-las num diretório que representa a intenção(que será nossa classe para a tarefa de classificação de intenção).</br>
<b>atenção</b>: O sistema foi excluído da coleta de utterances porque a resposta do sistema não possui intenção

In [3]:
import json

if __name__ == '__main__':
    with open("dialogues_018.json", 'r') as file:
        data_018 = json.load(file)

    for i in range(len(data_018)):
        frames_amount = len(data_018[i]['turns'])
        for j in range(frames_amount):
            current_frame = data_018[i]['turns'][j]
            speaker = current_frame['speaker']
            if speaker == 'USER':
                intent = current_frame['frames'][0]['state']['active_intent']
                utterance = current_frame['utterance']

                file_path = "dialogue_018/" + intent + "/" + str(i) + "_" + str(j) + ".txt"

                f = open(file_path, "w+")
                f.write(utterance)
                f.close()


Armazenar numa estrutura para treinamento no Keras

In [4]:
import tensorflow as tf

batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 1052 files belonging to 5 classes.
Using 842 files for training.


# Pre processamento de dados
Fazer uma bag-of-words

In [7]:
from keras.preprocessing.text import Tokenizer
import json

In [8]:
docs = []

with open("dialogues_018.json", 'r') as file:
    data_018 = json.load(file)

for i in range(len(data_018)):
    frames_amount = len(data_018[i]['turns'])
    for j in range(frames_amount):
        current_frame = data_018[i]['turns'][j]
        speaker = current_frame['speaker']
        if speaker == 'USER':
            intent = current_frame['frames'][0]['state']['active_intent']
            utterance = current_frame['utterance']

            docs.append(utterance)

In [10]:
t = Tokenizer()
t.fit_on_texts(docs)

# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

OrderedDict([('i', 397), ('need', 69), ('a', 120), ('one', 27), ('way', 10), ('flight', 224), ('and', 171), ('prefer', 24), ('traveling', 33), ('in', 52), ('premium', 8), ('economy', 59), ('class', 21), ('would', 67), ('like', 74), ('to', 345), ('leave', 59), ('next', 20), ('friday', 5), ('am', 93), ('nyc', 3), ('from', 155), ('seattle', 35), ('wa', 16), ('travel', 20), ('on', 233), ('delta', 13), ('airlines', 36), ('is', 129), ('this', 89), ('refundable', 43), ('ticket', 71), ('what', 57), ('the', 358), ('arrival', 13), ('airport', 59), ('sounds', 122), ('good', 114), ('me', 195), ('yes', 122), ('can', 125), ('you', 231), ('assist', 3), ('with', 19), ('buying', 3), ('that', 195), ('correct', 20), ('all', 77), ('needed', 8), ('thank', 75), ('help', 109), ('find', 58), ('flights', 113), ('for', 264), ('my', 50), ('upcoming', 1), ('trip', 63), ("i'll", 24), ('be', 99), ('going', 52), ('sf', 13), ('flying', 12), ('toronto', 2), ('canada', 2), ('4th', 14), ('of', 125), ('month', 48), ('are

In [22]:
from keras.preprocessing import text, sequence

In [31]:
seq_token = t.texts_to_sequences(docs)
train_data = sequence.pad_sequences(seq_token)

In [32]:
train_data.shape

(1052, 30)

# Classificação de intenções (Parte 1)
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM </br>
https://keras.io/examples/nlp/text_classification_with_transformer/ </br>

## LSTM

In [35]:
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.layers import Embedding
import keras.backend as K

model = Sequential()
model.add(Embedding(len(t.word_counts), 32))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss=['sparse_categorical_crossentropy'] , optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          14080     
                                                                 
 dense_8 (Dense)             (None, None, 32)          1056      
                                                                 
 dense_9 (Dense)             (None, None, 1)           33        
                                                                 
Total params: 15169 (59.25 KB)
Trainable params: 15169 (59.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(train_data, epochs=10)

## SVM

## CNN

## Transformer
implementar incialmente exatamente como está na documentação e depois testar mudar os parâmetros

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Implement a transformer block as a layer

In [6]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Implement embedding layer

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions